In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

def generate_green_llm_parameters():
    """
    Generate realistic parameters for the Green-LLM optimization model based on
    Google data center locations and real-world energy data.
    """
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Define data center locations
    datacenter_info = {
        0: {"name": "us-central1", "location": "Iowa", "renewable": 0.95, "co2": 430},
        1: {"name": "us-east1", "location": "South Carolina", "renewable": 0.29, "co2": 560},
        2: {"name": "us-east4", "location": "Northern Virginia", "renewable": 0.52, "co2": 322},
        3: {"name": "us-east5", "location": "Columbus", "renewable": 0.52, "co2": 322},
        4: {"name": "us-south1", "location": "Dallas", "renewable": 0.79, "co2": 321},
        5: {"name": "us-west1", "location": "Oregon", "renewable": 0.84, "co2": 94},
        6: {"name": "us-west2", "location": "Los Angeles", "renewable": 0.55, "co2": 198},
        7: {"name": "us-west3", "location": "Salt Lake City", "renewable": 0.29, "co2": 588},
        8: {"name": "us-west4", "location": "Las Vegas", "renewable": 0.26, "co2": 373}
    }
    


# Define electricity prices (cents/kWh) based on real average prices by state 
    population_by_region = {
        0: 3.2,    # Iowa
        1: 5.1,    # South Carolina 
        2: 8.6,    # Northern Virginia (using Virginia population)
        3: 11.8,   # Ohio (Columbus)
        4: 29.0,   # Texas (Dallas)
        5: 4.2,    # Oregon
        6: 39.5,   # California (Los Angeles)
        7: 3.3,    # Utah (Salt Lake City)
        8: 3.1     # Nevada (Las Vegas)
    }

# Define query popularity by type
    query_popularity = {
        0: 1.5,    # Completion - very common
        1: 2.0,    # Chat - most common
        2: 0.7,    # Embedding - less common
        3: 0.3,    # Fine-tuning - rare
        4: 0.8     # Inference - moderately common
    }
    
    # Define spatial and time dimensions
    I = list(range(9))  # 12 user areas
    J = list(range(9))   # 9 data centers (from Google)
    K = list(range(5))   # 5 query types (e.g., completion, chat, embedding, fine-tuning, inference)
    T = list(range(24))  # 24 time slots (hours in a day)
    R = list(range(3))   # 3 resource types (CPU, GPU, Memory)
    DeltaT = list(range(6)) # 6 time slots for water usage

    params = {}
    
    # Add sets
    params['I'] = I
    params['J'] = J
    params['K'] = K
    params['T'] = T
    params['R'] = R
    params['DeltaT'] = DeltaT
    
    # Create query demand distributions - make these time-of-day dependent
    # Different geographical areas will have different peak times
    
    # Base demand patterns by hour (24-hour cycle)
    hourly_patterns = np.array([
        0.3, 0.2, 0.15, 0.1, 0.1, 0.15, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95,  # 0-11
        1.0, 0.95, 0.9, 0.85, 0.8, 0.85, 0.9, 0.85, 0.75, 0.6, 0.5, 0.4  # 12-23
    ])
    
    # Create time zone offsets for different regions (simplified)
    # Assume I = 0-4: East, 5-9: Central, 10-14: Mountain, 15-19: Pacific
    time_offsets = np.zeros(len(I), dtype=int)
    time_offsets[0:5] = 0      # Eastern
    time_offsets[5:10] = 1     # Central 
    time_offsets[10:15] = 2    # Mountain
    time_offsets[15:20] = 3    # Pacific
    
    # Query parameters - using consistent ordering (i,k,t)
    lmbda = {}
    total_population = sum(population_by_region.values())
    base_demand = 30  # Base demand for a typical region

    for i in params['I']:
    # Calculate population factor (normalized to keep values in desired range)
        pop_factor = population_by_region[i] / total_population * len(params['I'])
    
        for k in params['K']:
            # Get popularity factor for this query type
            type_factor = query_popularity[k]
        
            for t in params['T']:
                # Time of day factor - business hours have more traffic
                local_hour = t
                if 9 <= local_hour < 18:  # Business hours
                    time_factor = 1.2
                elif 18 <= local_hour < 22:  # Evening
                    time_factor = 1.0
                else:  # Night/early morning
                    time_factor = 0.8
                
            # Calculate demand with small random variation
                raw_demand = base_demand * pop_factor * type_factor * time_factor
            
            # Add some randomness (±15%)
                variation = 0.85 + 0.3 * np.random.random()
            
            # Set final demand, ensuring it stays within 30-50 range
                final_demand = max(30, min(50, int(raw_demand * variation)))
                lmbda[(i, k, t)] = final_demand
    
    
    # Create a DataFrame for demand analysis
    demand_data = []
    for i in params['I']:
        for k in params['K']:
            for t in params['T']:
                demand_data.append({
                    'region': i, 
                    'region_name': list(datacenter_info.values())[i]['location'],
                    'query_type': k,
                    'hour': t,
                    'demand': lmbda[(i, k, t)],
                    'population': population_by_region[i]
                })


    # Token counts for different query types
    h = {
        0: 50,    # Completion - shorter prompts
        1: 150,   # Chat - medium prompts
        2: 500,   # Embedding - longer texts
        3: 1000,  # Fine-tuning - lots of examples
        4: 100    # Inference - medium prompts
    }
    params['h'] = h
    
    # Output token counts
    f = {
        0: 200,   # Completion - medium outputs
        1: 300,   # Chat - longer conversations
        2: 2000,    # Embedding - just vectors
        3: 100,   # Fine-tuning - model updates
        4: 500    # Inference - detailed outputs
    }
    params['f'] = f
    



    
    # Energy consumption coefficients
    tau_in = {k: 0.001 + 0.002 * np.random.random() for k in K}  # Input token energy
    tau_out = {k: 0.002 + 0.005 * np.random.random() for k in K}  # Output token energy (more computation)
    params['tau_in'] = tau_in
    params['tau_out'] = tau_out
    
    # Electricity prices - vary by location and time (peak/off-peak patterns)
    c = {}
    for j in J:
        base_price = electricity_prices[j] / 100.0  # Convert cents to dollars
        for t in T:
            # Apply time-of-day pricing (peak during business hours)
            if 8 <= t < 20:  # Peak hours
                factor = 1.2 + 0.3 * np.random.random()  # 20-50% premium
            else:  # Off-peak
                factor = 0.8 + 0.3 * np.random.random()  # 20-50% discount
            
            c[(j, t)] = base_price * factor
    params['c'] = c
    
    
    # Carbon intensity (kg CO2/kWh) - from Google data
    theta = {j: datacenter_info[j]['co2'] / 1000.0 for j in J}  # Convert g to kg
    params['theta'] = theta
    
    # Carbon tax ($/kg CO2)
    carbon_tax_base = 50.0  # $/ton = 0.05 $/kg
    delta = {(j, t): carbon_tax_base * (0.001 + 0.01 * np.random.random()) for j in J for t in T}
    params['delta'] = delta
    
    # Renewable energy availability (kW)
    P_w = {}
    for j in J:
        renewable_percentage = datacenter_info[j]['renewable']
        base_capacity = np.random.randint(2000, 5000)  # Base datacenter capacity
        
        for t in T:
            # Solar generation curve (for daytime hours)
            solar_factor = 0
            if 6 <= t < 18:  # Daylight hours
                # Peak at noon, lower at dawn/dusk
                hour_factor = 1.0 - abs(t - 12) / 6.0
                solar_factor = hour_factor * renewable_percentage
                
            # Wind generation (less predictable)
            wind_factor = 0.5 * renewable_percentage * np.random.random()
            
            # Total renewable factor
            total_factor = solar_factor + wind_factor
            
            # Final renewable capacity for this data center and time
            P_w[(j, t)] = base_capacity * total_factor
    params['P_w'] = P_w
    
    # Maximum power from grid (kW)
    P_max = {(j, t): np.random.randint(30000, 50000) for j in J for t in T}
    params['P_max'] = P_max
    
    
    # Power Usage Effectiveness
    # Modern data centers range from 1.1 (very efficient) to 1.5 (average)
    PUE = {j: 1.1 + 0.4 * np.random.random() for j in J}
    params['PUE'] = PUE
    
    # Resource capacity (units)
    C = {(r, j): np.random.randint(1000, 2000) for r in R for j in J}
    params['C'] = C
    
    # Resource requirements per query type
    alpha = {}
    for k in K:
        for r in R:
            # Different query types need different resources
            if r == 0:  # CPU
                alpha[(k, r)] = 0.01 + 0.05 * np.random.random()
            elif r == 1:  # GPU
                # Fine-tuning and large models need more GPU
                if k in [2, 3]:
                    alpha[(k, r)] = 0.05 + 0.15 * np.random.random()
                else:
                    alpha[(k, r)] = 0.01 + 0.05 * np.random.random()
            else:  # Memory
                alpha[(k, r)] = 0.02 + 0.08 * np.random.random()
    params['alpha'] = alpha
    
    # Previous model placement
    # Initially have some models placed but not all
    z_prev = {}
    for j in J:
        for k in K:
            for t in T:
                if t == 0:  # Initial placement
                    z_prev[(j, k, t)] = np.random.randint(0, 2)
                else:
                    # For subsequent time slots, we'd use optimization output
                    # But for t=1, just use random placement
                    z_prev[(j, k, t)] = z_prev[(j, k, 0)]
    params['z_prev'] = z_prev
    
    # Model download cost (units)
    f_download = {j: 10 + 20 * np.random.random() for j in J}
    params['f_download'] = f_download
    


    s = {}
    for i in I:
        for k in K:
            for t in T:
                # Random penalty between 10 and 15
                s[(i, k, t)] = 10 + 5 * np.random.random()
    params['s'] = s

    
    # Maximum unmet demand ratio
    Gamma = {i: 0.2 + 0.2 * np.random.random() for i in I}
    params['Gamma'] = Gamma
    
    # Error rates
    e = {
        0: 0.02,  # Completion
        1: 0.03,  # Chat
        2: 0.01,  # Embedding
        3: 0.05,  # Fine-tuning
        4: 0.02   # Inference
    }
    params['e'] = e
    
    # Minimum accuracy
    E = {k: 0.60 + 0.08 * np.random.random() for k in K}
    params['E'] = E
    
    # Token size (KB)
    beta = {(i, k, t): 0.5 + 1.5 * np.random.random() for i in I for k in K for t in T}
    params['beta'] = beta
    
    # Network bandwidth (Mbps)
    B = {}
    for i in I:
        for j in J:
            # Distance factor: closer regions have higher bandwidth
            region_i = i // 5  # 0=East, 1=Central, 2=Mountain, 3=West
            region_j = 0
            if j in [0]:
                region_j = 1  # Central
            elif j in [1, 2, 3]:
                region_j = 0  # East
            elif j in [4]:
                region_j = 1  # South (treat as Central)
            else:
                region_j = 3  # West
                
            distance = abs(region_i - region_j)
            base_bw = 20000.0  # 1 Gbps base
            
            # Further distances have more bandwidth variability and lower average
            bandwidth = base_bw * (1.0 - 0.1 * distance) * (0.7 + 0.6 * np.random.random())
            B[(i, j)] = max(500, bandwidth)  # Minimum 100 Mbps
    params['B'] = B
    
    # Network delay (s)
    d = {}
    for i in I:
        for j in J:
            # Similar to bandwidth, but inverse relationship
            region_i = i // 5
            region_j = 0
            if j in [0]:
                region_j = 1
            elif j in [1, 2, 3]:
                region_j = 0
            elif j in [4]:
                region_j = 1
            else:
                region_j = 3
                
            distance = abs(region_i - region_j)
            
            # Base latency plus distance factor (in seconds)
            delay = 0.01 + 0.01 * distance + 0.02 * np.random.random()
            d[(i, j)] = delay
    params['d'] = d
    
    # Processing delay (s/token)
    v = {(j, t): 0.001 + 0.002 * np.random.random() for j in J for t in T}
    params['v'] = v
    
    # Delay threshold (s)
    Delta = {}
    for i in I:
        for k in K:
            # Different thresholds for different query types
            base_threshold = 0.5
            if k in [2, 3]:  # Embedding and fine-tuning can take longer
                base_threshold = 2.0
            elif k == 4:  # Inference
                base_threshold = 1.0
                
            for t in T:
                Delta[(i, k, t)] = base_threshold * (0.8 + 0.4 * np.random.random())
    params['Delta'] = Delta
    
    # Water Usage Effectiveness
    WUE = {(j, t): 0.2 + 0.3 * np.random.random() for j in J for t in T}
    params['WUE'] = WUE
    
    # Energy-Water Intensity Factor
    EWIF = {(j, t): 0.01 + 0.01 * np.random.random() for j in J for t in T}
    params['EWIF'] = EWIF
    
    # Maximum water consumption
    Z = np.random.uniform(8000, 12000)
    params['Z'] = Z
    return params
    
# Generate parameters
params = generate_green_llm_parameters()

# Summary of key parameters
# print(f"Number of user areas: {len(params['I'])}")
# print(f"Number of data centers: {len(params['J'])}")
# print(f"Number of query types: {len(params['K'])}")
# print(f"Number of time slots: {len(params['T'])}")
# print(f"Number of resource types: {len(params['R'])}")

# Sample of electricity prices ($/kWh)
# print("\nSample electricity prices ($/kWh):")
# for j in params['J']:
#     for t in range(0, 24, 8):
#         print(f"Data center {j} at time {t}: ${params['c'][(j, t)]:.4f}")

# Sample of CO2 intensities
# print("\nCO2 intensities (kg/kWh):")
# for j in params['J']:
#     print(f"Data center {j} ({list(params['theta'].values())[j]:.4f} kg/kWh)")

# # Sample of renewable energy availability
# print("\nSample renewable energy availability (kW):")
# for j in params['J']:
#     for t in range(0, 24, 8):
#         print(f"Data center {j} at time {t}: {params['P_w'][(j, t)]:.2f} kW")

# Sample of demand
total_demand_by_hour = {t: 0 for t in params['T']}
for i in params['I']:
    for k in params['K']:
        for t in params['T']:
            total_demand_by_hour[t] += params['lambda'][(i, k, t)]

total_demand_by_hour
# Function to save parameters to file if needed
# def save_params_to_file(params, filename='green_llm_params.py'):
#     with open(filename, 'w') as f:
#         f.write("# Auto-generated Green-LLM parameters\n\n")
#         f.write("def get_parameters():\n")
#         f.write("    params = {}\n\n")
        
#         # Write basic sets
#         for set_name in ['I', 'J', 'K', 'T', 'R']:
#             f.write(f"    params['{set_name}'] = {params[set_name]}\n")
        
#         # Write dictionaries
#         for key, value in params.items():
#             if key not in ['I', 'J', 'K', 'T', 'R']:
#                 f.write(f"\n    # {key}\n")
#                 f.write(f"    params['{key}'] = {value}\n")
        
#         f.write("\n    return params\n")

# Uncomment to save to file
# save_params_to_file(params)

# Return parameters for use in optimization


NameError: name 'electricity_prices' is not defined

In [1]:
import pandas as pd
import numpy as np
import os

def extract_electricity_data(filepath, output_dir='extracted_data'):
    """
    Extract electricity price data from 2020-2024 and save in various formats.
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file containing electricity data
    output_dir : str
        Directory to save extracted data files
    
    Returns:
    --------
    dict
        Dictionary containing the extracted DataFrames
    """

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Read the CSV file
    print(f"Reading data from {filepath}...")
    df = pd.read_csv(filepath)
    
    # Filter for 2020-2024
    df_filtered = df[(df['year'] >= 2020) & (df['year'] <= 2024)].copy()
    print(f"Filtered data to {len(df_filtered)} rows from 2020-2024.")
    
    # Create output dictionary
    output_data = {}
    
    # 1. All sectors data (main dataset for analysis)
    all_sectors = df_filtered[df_filtered['sectorName'] == 'all sectors'].copy()
    output_data['all_sectors'] = all_sectors
    
    # Save all sectors data
    all_sectors.to_csv(f"{output_dir}/electricity_all_sectors_2020_2024.csv", index=False)
    all_sectors.to_excel(f"{output_dir}/electricity_all_sectors_2020_2024.xlsx", index=False)
    
    # 2. Calculate state summaries
    print("Calculating state summaries...")
    state_summaries = []
    
    for state in all_sectors['stateDescription'].unique():
        state_data = all_sectors[all_sectors['stateDescription'] == state]
        
        # Calculate average price, total revenue, etc.
        avg_price = state_data['price'].mean()
        total_revenue = state_data['revenue'].sum()
        total_sales = state_data['sales'].sum()
        avg_customers = state_data['customers'].mean()
        
        state_summaries.append({
            'state': state,
            'avg_price': round(avg_price, 2),
            'total_revenue': round(total_revenue, 2),
            'total_sales': round(total_sales, 2),
            'avg_customers': round(avg_customers) if not np.isnan(avg_customers) else None
        })
    
    # Convert to DataFrame and sort by price
    state_summary_df = pd.DataFrame(state_summaries)
    state_summary_df = state_summary_df.sort_values('avg_price', ascending=False)
    output_data['state_summary'] = state_summary_df
    
    # Save state summary
    state_summary_df.to_csv(f"{output_dir}/state_summary_2020_2024.csv", index=False)
    state_summary_df.to_excel(f"{output_dir}/state_summary_2020_2024.xlsx", index=False)
    
    # 3. Calculate yearly trends by state
    print("Calculating yearly trends by state...")
    yearly_trends = []
    
    for state in all_sectors['stateDescription'].unique():
        state_data = all_sectors[all_sectors['stateDescription'] == state]
        
        for year in range(2020, 2025):
            year_data = state_data[state_data['year'] == year]
            
            if not year_data.empty:
                avg_price = year_data['price'].mean()
                
                yearly_trends.append({
                    'state': state,
                    'year': year,
                    'avg_price': round(avg_price, 2)
                })
    
    yearly_trends_df = pd.DataFrame(yearly_trends)
    output_data['yearly_trends'] = yearly_trends_df
    
    # Save yearly trends
    yearly_trends_df.to_csv(f"{output_dir}/yearly_trends_2020_2024.csv", index=False)
    yearly_trends_df.to_excel(f"{output_dir}/yearly_trends_2020_2024.xlsx", index=False)
    
    # 4. Identify top and bottom states
    top_states = state_summary_df.head(10).copy()
    bottom_states = state_summary_df.tail(10).copy()
    
    output_data['top_states'] = top_states
    output_data['bottom_states'] = bottom_states
    
    # Save top/bottom states
    top_states.to_csv(f"{output_dir}/top_price_states.csv", index=False)
    bottom_states.to_csv(f"{output_dir}/bottom_price_states.csv", index=False)
    
    # 5. Save data in JSON format (for web applications)
    print("Saving data in JSON format...")
    state_summary_df.to_json(f"{output_dir}/state_summary.json", orient='records')
    yearly_trends_df.to_json(f"{output_dir}/yearly_trends.json", orient='records')
    
    # 6. Create a pivot table for easier analysis
    pivot_df = yearly_trends_df.pivot(index='state', columns='year', values='avg_price')
    pivot_df.to_csv(f"{output_dir}/price_pivot_table.csv")
    pivot_df.to_excel(f"{output_dir}/price_pivot_table.xlsx")
    output_data['pivot_table'] = pivot_df
    
    print(f"Data extraction complete. Files saved to {output_dir}/")
    return output_data

if __name__ == "__main__":
    # Usage example
    # 1. Check if the file exists
    file_path = r"C:\Users\jiami\ASU Dropbox\Jiaming Cheng\Network_letter\clean_data.csv"
    print(f"File exists: {os.path.exists(file_path)}")

    # 2. Check the current working directory
    print(f"Current working directory: {os.getcwd()}")

    # 3. List files in the current directory
    print("Files in current directory:")
    for file in os.listdir():
        print(f"  - {file}")

    # 4. Try with a relative path if the file is in the same folder as your notebook
    if "clean_data.csv" in os.listdir():
        print("Found clean_data.csv in current directory, trying with relative path...")
        extracted_data = extract_electricity_data("clean_data.csv")

File exists: True
Current working directory: c:\Users\jiami\ASU Dropbox\Jiaming Cheng\Network_letter
Files in current directory:
  - avgprice_annual.xlsx
  - clean_data.csv
  - clean_data.csv.zip
  - dataset
  - demand_plots.png
  - Green_LLM.py
  - green_llm_model.py
  - Green_LLM_opt.py
  - model_iis.ilp
  - model_test.ipynb
  - test1.ipynb
Found clean_data.csv in current directory, trying with relative path...
Reading data from clean_data.csv...
Filtered data to 15190 rows from 2020-2024.
Calculating state summaries...
Calculating yearly trends by state...
Saving data in JSON format...
Data extraction complete. Files saved to extracted_data/


In [4]:
import pandas as pd
import numpy as np
import os
import json
import openpyxl


def extract_electricity_data(filepath, output_dir='extracted_data'):
    """
    Extract electricity price data from 2020-2024 and save in various formats.
    Preserves monthly data for all states.
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file containing electricity data
    output_dir : str
        Directory to save extracted data files
    
    Returns:
    --------
    dict
        Dictionary containing the extracted DataFrames
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Read the CSV file
    print(f"Reading data from {filepath}...")
    df = pd.read_csv(filepath)
    
    # Filter for 2020-2024
    df_filtered = df[(df['year'] >= 2020) & (df['year'] <= 2024)].copy()
    print(f"Filtered data to {len(df_filtered)} rows from 2020-2024.")
    
    # Create output dictionary
    output_data = {}
    
    # 1. All sectors data (main dataset for analysis)
    all_sectors = df_filtered[df_filtered['sectorName'] == 'all sectors'].copy()
    output_data['all_sectors'] = all_sectors
    
    # Save all sectors data (preserves monthly data)
    all_sectors.to_csv(f"{output_dir}/electricity_all_sectors_2020_2024.csv", index=False)
    all_sectors.to_excel(f"{output_dir}/electricity_all_sectors_2020_2024.xlsx", index=False)
    
    # 2. Save monthly data by state (key requirement)
    print("Organizing monthly data by state...")
    
    # Create a subdirectory for state-specific data
    state_dir = f"{output_dir}/states"
    if not os.path.exists(state_dir):
        os.makedirs(state_dir)
    
    # Get list of all states
    all_states = sorted(all_sectors['stateDescription'].unique())
    
    # Create a dictionary to store monthly data by state
    monthly_data_by_state = {}
    
    for state in all_states:
        # Filter data for this state
        state_data = all_sectors[all_sectors['stateDescription'] == state].copy()
        
        # Sort by year and month
        state_data = state_data.sort_values(['year', 'month'])
        
        # Save to CSV and Excel
        state_data.to_csv(f"{state_dir}/{state.replace(' ', '_')}_monthly_2020_2024.csv", index=False)
        state_data.to_excel(f"{state_dir}/{state.replace(' ', '_')}_monthly_2020_2024.xlsx", index=False)
        
        # Store in dictionary
        monthly_data_by_state[state] = state_data
        
    output_data['monthly_by_state'] = monthly_data_by_state
    
    # 3. Create a comprehensive monthly dataset
    monthly_all = all_sectors.sort_values(['stateDescription', 'year', 'month']).copy()
    monthly_all.to_csv(f"{output_dir}/monthly_all_states_2020_2024.csv", index=False)
    monthly_all.to_excel(f"{output_dir}/monthly_all_states_2020_2024.xlsx", index=False)
    output_data['monthly_all'] = monthly_all
    
    # 4. Calculate state summaries
    print("Calculating state summaries...")
    state_summaries = []
    
    for state in all_states:
        state_data = all_sectors[all_sectors['stateDescription'] == state]
        
        # Calculate average price, total revenue, etc.
        avg_price = state_data['price'].mean()
        total_revenue = state_data['revenue'].sum()
        total_sales = state_data['sales'].sum()
        avg_customers = state_data['customers'].mean()
        
        # Calculate monthly statistics
        monthly_stats = []
        for year in range(2020, 2025):
            for month in range(1, 13):
                month_data = state_data[(state_data['year'] == year) & (state_data['month'] == month)]
                if not month_data.empty:
                    monthly_stats.append({
                        'year': year,
                        'month': month,
                        'price': float(month_data['price'].values[0]) if not month_data['price'].isna().all() else None,
                        'revenue': float(month_data['revenue'].values[0]) if not month_data['revenue'].isna().all() else None,
                        'sales': float(month_data['sales'].values[0]) if not month_data['sales'].isna().all() else None,
                        'customers': float(month_data['customers'].values[0]) if not month_data['customers'].isna().all() else None
                    })
        
        state_summaries.append({
            'state': state,
            'avg_price': round(avg_price, 2),
            'total_revenue': round(total_revenue, 2),
            'total_sales': round(total_sales, 2),
            'avg_customers': round(avg_customers) if not np.isnan(avg_customers) else None,
            'monthly_data': monthly_stats
        })
    
    # Convert to DataFrame (excluding monthly_data for DataFrame version)
    state_summary_df = pd.DataFrame([{k: v for k, v in summary.items() if k != 'monthly_data'} 
                                     for summary in state_summaries])
    state_summary_df = state_summary_df.sort_values('avg_price', ascending=False)
    output_data['state_summary'] = state_summary_df
    
    # Save state summary
    state_summary_df.to_csv(f"{output_dir}/state_summary_2020_2024.csv", index=False)
    state_summary_df.to_excel(f"{output_dir}/state_summary_2020_2024.xlsx", index=False)
    
    # Save the full summary with monthly data as JSON
    with open(f"{output_dir}/state_summary_with_monthly.json", 'w') as f:
        json.dump(state_summaries, f, indent=2)
    
    # 5. Calculate monthly trends for all states
    print("Calculating monthly trends...")
    monthly_trends = []
    
    for year in range(2020, 2025):
        for month in range(1, 13):
            month_data = all_sectors[(all_sectors['year'] == year) & (all_sectors['month'] == month)]
            
            if not month_data.empty:
                avg_price = month_data['price'].mean()
                
                monthly_trends.append({
                    'year': year,
                    'month': month,
                    'date': f"{year}-{month:02d}",
                    'avg_price': round(avg_price, 2)
                })
    
    monthly_trends_df = pd.DataFrame(monthly_trends)
    output_data['monthly_trends'] = monthly_trends_df
    
    # Save monthly trends
    monthly_trends_df.to_csv(f"{output_dir}/monthly_trends_2020_2024.csv", index=False)
    monthly_trends_df.to_excel(f"{output_dir}/monthly_trends_2020_2024.xlsx", index=False)
    
    # 6. Calculate yearly trends by state
    print("Calculating yearly trends by state...")
    yearly_trends = []
    
    for state in all_states:
        state_data = all_sectors[all_sectors['stateDescription'] == state]
        
        for year in range(2020, 2025):
            year_data = state_data[state_data['year'] == year]
            
            if not year_data.empty:
                avg_price = year_data['price'].mean()
                
                yearly_trends.append({
                    'state': state,
                    'year': year,
                    'avg_price': round(avg_price, 2)
                })
    
    yearly_trends_df = pd.DataFrame(yearly_trends)
    output_data['yearly_trends'] = yearly_trends_df
    
    # Save yearly trends
    yearly_trends_df.to_csv(f"{output_dir}/yearly_trends_2020_2024.csv", index=False)
    yearly_trends_df.to_excel(f"{output_dir}/yearly_trends_2020_2024.xlsx", index=False)
    
    # 7. Identify top and bottom states
    top_states = state_summary_df.head(10).copy()
    bottom_states = state_summary_df.tail(10).copy()
    
    output_data['top_states'] = top_states
    output_data['bottom_states'] = bottom_states
    
    # Save top/bottom states
    top_states.to_csv(f"{output_dir}/top_price_states.csv", index=False)
    bottom_states.to_csv(f"{output_dir}/bottom_price_states.csv", index=False)
    
    # 8. Save data in JSON format (for web applications)
    print("Saving data in JSON format...")
    state_summary_df.to_json(f"{output_dir}/state_summary.json", orient='records')
    yearly_trends_df.to_json(f"{output_dir}/yearly_trends.json", orient='records')
    monthly_trends_df.to_json(f"{output_dir}/monthly_trends.json", orient='records')
    
    # 9. Create pivot tables for easier analysis
    # Yearly pivot table
    yearly_pivot_df = yearly_trends_df.pivot(index='state', columns='year', values='avg_price')
    yearly_pivot_df.to_csv(f"{output_dir}/yearly_price_pivot_table.csv")
    yearly_pivot_df.to_excel(f"{output_dir}/yearly_price_pivot_table.xlsx")
    output_data['yearly_pivot'] = yearly_pivot_df
    
    # Monthly pivot table
    monthly_pivot = pd.pivot_table(
        all_sectors,
        values='price',
        index=['stateDescription'],
        columns=['year', 'month'],
        aggfunc='mean'
    )
    monthly_pivot.to_csv(f"{output_dir}/monthly_price_pivot_table.csv")
    monthly_pivot.to_excel(f"{output_dir}/monthly_price_pivot_table.xlsx")
    output_data['monthly_pivot'] = monthly_pivot
    
    print(f"Data extraction complete. Files saved to {output_dir}/")
    return output_data

if __name__ == "__main__":
    # Usage example
    extract_electricity_data('clean_data.csv')

Reading data from clean_data.csv...
Filtered data to 15190 rows from 2020-2024.
Organizing monthly data by state...
Calculating state summaries...
Calculating monthly trends...
Calculating yearly trends by state...
Saving data in JSON format...
Data extraction complete. Files saved to extracted_data/


In [13]:
import pandas as pd
import numpy as np
import kaleido
def add_rto_to_electricity_data(input_file, output_file):
    """
    Add RTO/ISO information to electricity data based on state location.
    
    Parameters:
    -----------
    input_file : str
        Path to the input Excel file containing electricity data
    output_file : str
        Path where the modified Excel file will be saved
    """
    print(f"Reading electricity data from {input_file}...")
    
    # Read the Excel file
    df = pd.read_excel(input_file)
    
    # Print dataset info
    print(f"Loaded {len(df)} rows of data")
    print("Sample of first 3 rows:")
    print(df.head(3))
    
    # Create a mapping of states to RTOs/ISOs
    # This mapping is based on the RTO/ISO information provided
    state_to_rto_mapping = {
        # CAISO (California ISO)
        "California": "CAISO",
        
        # ERCOT (Electric Reliability Council of Texas)
        "Texas": "ERCOT",
        
        # PJM Interconnection
        "Delaware": "PJM",
        "Illinois": "PJM",  # Note: Parts of Illinois are in MISO too
        "Indiana": "PJM",   # Note: Parts of Indiana are in MISO too
        "Kentucky": "PJM",  # Note: Parts of Kentucky are in MISO too
        "Maryland": "PJM",
        "Michigan": "PJM",  # Note: Parts of Michigan are in MISO too
        "New Jersey": "PJM",
        "North Carolina": "PJM",  # Note: Only parts of NC are in PJM
        "Ohio": "PJM",
        "Pennsylvania": "PJM",
        "Tennessee": "PJM",  # Note: Only parts of TN are in PJM
        "Virginia": "PJM",
        "West Virginia": "PJM",
        "District of Columbia": "PJM",
        
        # MISO (Midcontinent Independent System Operator)
        "Minnesota": "MISO",
        "Iowa": "MISO",
        "Missouri": "MISO",
        "Wisconsin": "MISO",
        "Michigan": "MISO",  # Note: Overlap with PJM
        "Illinois": "MISO",  # Note: Overlap with PJM
        "Indiana": "MISO",   # Note: Overlap with PJM
        "Kentucky": "MISO",  # Note: Overlap with PJM
        "Arkansas": "MISO",  # Note: Parts of Arkansas are in SPP
        "Mississippi": "MISO",
        "Louisiana": "MISO",
        "North Dakota": "MISO",
        "South Dakota": "MISO",
        
        # SPP (Southwest Power Pool)
        "Oklahoma": "SPP",
        "Kansas": "SPP",
        "Nebraska": "SPP",
        "Arkansas": "SPP",    # Note: Overlap with MISO
        "Louisiana": "SPP",   # Note: Overlap with MISO
        "Missouri": "SPP",    # Note: Overlap with MISO
        "New Mexico": "SPP",
        
        # NYISO (New York ISO)
        "New York": "NYISO",
        
        # ISO-NE (ISO New England)
        "Connecticut": "ISO-NE",
        "Maine": "ISO-NE",
        "Massachusetts": "ISO-NE",
        "New Hampshire": "ISO-NE",
        "Rhode Island": "ISO-NE",
        "Vermont": "ISO-NE",
        
        # WEIM (Western Energy Imbalance Market)
        "Arizona": "WEIM",
        "California": "WEIM",  # Note: Overlap with CAISO
        "Idaho": "WEIM",
        "Nevada": "WEIM",
        "Oregon": "WEIM",
        "Utah": "WEIM",
        "Washington": "WEIM",
        "Wyoming": "WEIM",
        "Colorado": "WEIM",
        "Montana": "WEIM"
    }
    
    # Function to handle overlapping RTOs (taking primary RTO for simplicity)
    def get_primary_rto(state):
        # For states with overlapping RTOs, prioritize as follows:
        # CAISO > ERCOT > PJM > MISO > SPP > NYISO > ISO-NE > WEIM
        if state == "California":
            return "CAISO"  # Prioritize CAISO over WEIM for California
        elif state == "Texas":
            return "ERCOT"  # Most of Texas is ERCOT
        # For states in both PJM and MISO, prioritize based on which covers more of the state
        elif state in ["Illinois", "Michigan"]:
            return "MISO"   # More of these states are in MISO
        elif state in ["Indiana", "Kentucky"]:
            return "PJM"    # More of these states are in PJM
        elif state == "Arkansas":
            return "SPP"    # More of Arkansas is in SPP
        elif state == "Louisiana":
            return "MISO"   # More of Louisiana is in MISO
        elif state == "Missouri":
            return "MISO"   # More of Missouri is in MISO
        else:
            return state_to_rto_mapping.get(state, "Other")
    
    # Add RTO/ISO column
    print("Adding RTO/ISO information...")
    df['RTO_ISO'] = df['stateDescription'].apply(get_primary_rto)
    
    # Handle regional entries
    # For regional entries, set RTO/ISO based on the region name or mark as "Multiple"
    region_to_rto = {
        "New England": "ISO-NE",
        "Middle Atlantic": "Multiple",  # Covers PJM and NYISO areas
        "East North Central": "Multiple",  # Covers PJM and MISO areas
        "West North Central": "Multiple",  # Covers MISO and SPP areas
        "South Atlantic": "Multiple",  # Covers PJM areas
        "East South Central": "Multiple",  # Covers PJM and MISO areas
        "West South Central": "Multiple",  # Covers ERCOT, SPP, and MISO areas
        "Mountain": "Multiple",  # Covers WEIM areas
        "Pacific Contiguous": "Multiple",  # Covers CAISO and WEIM areas
        "Pacific Noncontiguous": "Other",  # Alaska and Hawaii are not part of the major RTOs/ISOs
        "U.S. Total": "Multiple"
    }
    
    # Update regional entries
    for region, rto in region_to_rto.items():
        df.loc[df['stateDescription'] == region, 'RTO_ISO'] = rto
    
    # Count entries by RTO/ISO
    rto_counts = df['RTO_ISO'].value_counts()
    print("\nNumber of entries by RTO/ISO:")
    print(rto_counts)
    
    # Save modified data
    print(f"\nSaving modified data to {output_file}...")
    df.to_excel(output_file, index=False)
    
    print("Done!")
    return df

# Usage example
if __name__ == "__main__":
    input_file = "monthly_all_states_2020_2024.xlsx"
    output_file = "monthly_all_states_with_rto_2020_2024.xlsx"
    
    processed_data = add_rto_to_electricity_data(input_file, output_file)
    
    # Display sample of the modified data
    print("\nSample of modified data:")
    print(processed_data[['year', 'month', 'stateDescription', 'RTO_ISO', 'price']].head(10))

Reading electricity data from monthly_all_states_2020_2024.xlsx...
Loaded 3038 rows of data
Sample of first 3 rows:
   year  month stateDescription   sectorName  customers  price    revenue  \
0  2020      1          Alabama  all sectors    2639176   9.53  683.99992   
1  2020      2          Alabama  all sectors    2639076   9.44  635.71477   
2  2020      3          Alabama  all sectors    2648998   9.34  605.21129   

        sales  
0  7176.42851  
1  6737.74734  
2  6479.78199  
Adding RTO/ISO information...

Number of entries by RTO/ISO:
RTO_ISO
PJM         588
MISO        490
WEIM        441
Multiple    441
Other       343
ISO-NE      343
SPP         245
CAISO        49
NYISO        49
ERCOT        49
Name: count, dtype: int64

Saving modified data to monthly_all_states_with_rto_2020_2024.xlsx...
Done!

Sample of modified data:
   year  month stateDescription RTO_ISO  price
0  2020      1          Alabama   Other   9.53
1  2020      2          Alabama   Other   9.44
2  2020     

In [16]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from tabulate import tabulate
from datetime import datetime
import kaleido

def run_rto_analysis_pipeline(input_file='monthly_all_states_2020_2024.xlsx', 
                             output_dir='rto_analysis'):
    """
    Complete analysis pipeline to process electricity data, add RTO/ISO mapping,
    generate visualizations, and create an analysis report.
    
    Parameters:
    -----------
    input_file : str
        Path to the input Excel file containing electricity data
    output_dir : str
        Directory where all output files will be saved
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    # Create directories for different output types
    data_dir = os.path.join(output_dir, 'data')
    figures_dir = os.path.join(output_dir, 'figures')
    report_dir = os.path.join(output_dir, 'reports')
    
    for directory in [data_dir, figures_dir, report_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)
    
    # Step 1: Add RTO/ISO mapping to the data
    print("Step 1: Adding RTO/ISO mapping to the data...")
    modified_data = add_rto_to_electricity_data(
        input_file, 
        os.path.join(data_dir, 'monthly_all_states_with_rto.xlsx')
    )
    
    # Step 2: Generate visualizations
    print("\nStep 2: Generating visualizations...")
    visualizations = visualize_rto_analysis(
        os.path.join(data_dir, 'monthly_all_states_with_rto.xlsx'),
        figures_dir
    )
    
    # Step 3: Generate analysis report
    print("\nStep 3: Generating analysis report...")
    report = generate_rto_analysis_report(
        os.path.join(data_dir, 'monthly_all_states_with_rto.xlsx'),
        os.path.join(report_dir, 'rto_analysis_report.md')
    )
    
    # Step 4: Generate dashboard data
    print("\nStep 4: Preparing data for dashboard...")
    prepare_dashboard_data(
        modified_data,
        os.path.join(data_dir, 'dashboard_data')
    )
    
    print(f"\nAnalysis complete! All outputs saved to {output_dir}")
    return {
        'modified_data': modified_data,
        'visualizations': visualizations,
        'report': report
    }

def add_rto_to_electricity_data(input_file, output_file):
    """
    Add RTO/ISO information to electricity data based on state location.
    
    Parameters:
    -----------
    input_file : str
        Path to the input Excel file containing electricity data
    output_file : str
        Path where the modified Excel file will be saved
        
    Returns:
    --------
    pandas.DataFrame
        The modified DataFrame with RTO/ISO information
    """
    print(f"Reading electricity data from {input_file}...")
    
    # Read the Excel file
    df = pd.read_excel(input_file)
    
    # Print dataset info
    print(f"Loaded {len(df)} rows of data")
    
    # Create a mapping of states to RTOs/ISOs
    # This mapping is based on the RTO/ISO information provided
    state_to_rto_mapping = {
        # CAISO (California ISO)
        "California": "CAISO",
        
        # ERCOT (Electric Reliability Council of Texas)
        "Texas": "ERCOT",
        
        # PJM Interconnection
        "Delaware": "PJM",
        "Illinois": "PJM",  # Note: Parts of Illinois are in MISO too
        "Indiana": "PJM",   # Note: Parts of Indiana are in MISO too
        "Kentucky": "PJM",  # Note: Parts of Kentucky are in MISO too
        "Maryland": "PJM",
        "Michigan": "PJM",  # Note: Parts of Michigan are in MISO too
        "New Jersey": "PJM",
        "North Carolina": "PJM",  # Note: Only parts of NC are in PJM
        "Ohio": "PJM",
        "Pennsylvania": "PJM",
        "Tennessee": "PJM",  # Note: Only parts of TN are in PJM
        "Virginia": "PJM",
        "West Virginia": "PJM",
        "District of Columbia": "PJM",
        
        # MISO (Midcontinent Independent System Operator)
        "Minnesota": "MISO",
        "Iowa": "MISO",
        "Missouri": "MISO",
        "Wisconsin": "MISO",
        "Michigan": "MISO",  # Note: Overlap with PJM
        "Illinois": "MISO",  # Note: Overlap with PJM
        "Indiana": "MISO",   # Note: Overlap with PJM
        "Kentucky": "MISO",  # Note: Overlap with PJM
        "Arkansas": "MISO",  # Note: Parts of Arkansas are in SPP
        "Mississippi": "MISO",
        "Louisiana": "MISO",
        "North Dakota": "MISO",
        "South Dakota": "MISO",
        
        # SPP (Southwest Power Pool)
        "Oklahoma": "SPP",
        "Kansas": "SPP",
        "Nebraska": "SPP",
        "Arkansas": "SPP",    # Note: Overlap with MISO
        "Louisiana": "SPP",   # Note: Overlap with MISO
        "Missouri": "SPP",    # Note: Overlap with MISO
        "New Mexico": "SPP",
        
        # NYISO (New York ISO)
        "New York": "NYISO",
        
        # ISO-NE (ISO New England)
        "Connecticut": "ISO-NE",
        "Maine": "ISO-NE",
        "Massachusetts": "ISO-NE",
        "New Hampshire": "ISO-NE",
        "Rhode Island": "ISO-NE",
        "Vermont": "ISO-NE",
        
        # WEIM (Western Energy Imbalance Market)
        "Arizona": "WEIM",
        "California": "WEIM",  # Note: Overlap with CAISO
        "Idaho": "WEIM",
        "Nevada": "WEIM",
        "Oregon": "WEIM",
        "Utah": "WEIM",
        "Washington": "WEIM",
        "Wyoming": "WEIM",
        "Colorado": "WEIM",
        "Montana": "WEIM"
    }
    
    # Function to handle overlapping RTOs (taking primary RTO for simplicity)
    def get_primary_rto(state):
        # For states with overlapping RTOs, prioritize as follows:
        # CAISO > ERCOT > PJM > MISO > SPP > NYISO > ISO-NE > WEIM
        if state == "California":
            return "CAISO"  # Prioritize CAISO over WEIM for California
        elif state == "Texas":
            return "ERCOT"  # Most of Texas is ERCOT
        # For states in both PJM and MISO, prioritize based on which covers more of the state
        elif state in ["Illinois", "Michigan"]:
            return "MISO"   # More of these states are in MISO
        elif state in ["Indiana", "Kentucky"]:
            return "PJM"    # More of these states are in PJM
        elif state == "Arkansas":
            return "SPP"    # More of Arkansas is in SPP
        elif state == "Louisiana":
            return "MISO"   # More of Louisiana is in MISO
        elif state == "Missouri":
            return "MISO"   # More of Missouri is in MISO
        else:
            return state_to_rto_mapping.get(state, "Other")
    
    # Add RTO/ISO column
    print("Adding RTO/ISO information...")
    df['RTO_ISO'] = df['stateDescription'].apply(get_primary_rto)
    
    # Handle regional entries
    # For regional entries, set RTO/ISO based on the region name or mark as "Multiple"
    region_to_rto = {
        "New England": "ISO-NE",
        "Middle Atlantic": "Multiple",  # Covers PJM and NYISO areas
        "East North Central": "Multiple",  # Covers PJM and MISO areas
        "West North Central": "Multiple",  # Covers MISO and SPP areas
        "South Atlantic": "Multiple",  # Covers PJM areas
        "East South Central": "Multiple",  # Covers PJM and MISO areas
        "West South Central": "Multiple",  # Covers ERCOT, SPP, and MISO areas
        "Mountain": "Multiple",  # Covers WEIM areas
        "Pacific Contiguous": "Multiple",  # Covers CAISO and WEIM areas
        "Pacific Noncontiguous": "Other",  # Alaska and Hawaii are not part of the major RTOs/ISOs
        "U.S. Total": "Multiple"
    }
    
    # Update regional entries
    for region, rto in region_to_rto.items():
        df.loc[df['stateDescription'] == region, 'RTO_ISO'] = rto
    
    # Count entries by RTO/ISO
    rto_counts = df['RTO_ISO'].value_counts()
    print("\nNumber of entries by RTO/ISO:")
    print(rto_counts)
    
    # Save modified data
    print(f"\nSaving modified data to {output_file}...")
    df.to_excel(output_file, index=False)
    
    return df

def visualize_rto_analysis(input_file, output_dir='figures'):
    """
    Create visualizations to analyze electricity prices by RTO/ISO.
    
    Parameters:
    -----------
    input_file : str
        Path to the Excel file containing electricity data with RTO/ISO information
    output_dir : str
        Directory where visualization files will be saved
        
    Returns:
    --------
    dict
        Dictionary containing the visualization figures
    """
    print(f"Reading data from {input_file}...")
    
    # Read the Excel file
    df = pd.read_excel(input_file)
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Filter out rows not relevant to this analysis (e.g., non-"all sectors" data)
    if 'sectorName' in df.columns:
        df = df[df['sectorName'] == 'all sectors'].copy()
    
    # Add date column for time series analysis
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')
    
    # 1. Average price by RTO/ISO
    print("Creating average price by RTO/ISO visualization...")
    rto_avg_price = df.groupby('RTO_ISO')['price'].mean().reset_index()
    rto_avg_price = rto_avg_price.sort_values('price', ascending=False)
    
    fig1 = px.bar(
        rto_avg_price,
        x='RTO_ISO',
        y='price',
        title='Average Electricity Price by RTO/ISO (2020-2024)',
        labels={'price': 'Average Price ($/kWh)', 'RTO_ISO': 'RTO/ISO'},
        color='price',
        color_continuous_scale='Blues',
        height=600
    )
    
    fig1.update_layout(xaxis_tickangle=-45)
    fig1.write_html(os.path.join(output_dir, 'avg_price_by_rto.html'))
    fig1.write_image(os.path.join(output_dir, 'avg_price_by_rto.png'))
    
    # Get monthly average prices by RTO/ISO for subsequent visualizations
    monthly_rto_prices = df.groupby(['RTO_ISO', 'date'])['price'].mean().reset_index()
    
    # Filter to major RTOs only (exclude "Multiple" and "Other")
    major_rtos = ['CAISO', 'ERCOT', 'ISO-NE', 'MISO', 'NYISO', 'PJM', 'SPP', 'WEIM']
    monthly_major_rtos = monthly_rto_prices[monthly_rto_prices['RTO_ISO'].isin(major_rtos)]
    
    # 2. Price trends over time by RTO/ISO
    print("Creating price trends by RTO/ISO visualization...")
    fig2 = px.line(
        monthly_major_rtos,
        x='date',
        y='price',
        color='RTO_ISO',
        title='Monthly Electricity Price Trends by RTO/ISO (2020-2024)',
        labels={'price': 'Price ($/kWh)', 'date': 'Date', 'RTO_ISO': 'RTO/ISO'},
        height=600
    )
    
    fig2.update_layout(
        hovermode="x unified",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    fig2.write_html(os.path.join(output_dir, 'price_trends_by_rto.html'))
    fig2.write_image(os.path.join(output_dir, 'price_trends_by_rto.png'))
    
    # 3. Price volatility by RTO/ISO
    print("Creating price volatility by RTO/ISO visualization...")
    rto_price_std = df.groupby('RTO_ISO')['price'].std().reset_index()
    rto_price_std = rto_price_std.sort_values('price', ascending=False)
    
    fig3 = px.bar(
        rto_price_std,
        x='RTO_ISO',
        y='price',
        title='Electricity Price Volatility by RTO/ISO (2020-2024)',
        labels={'price': 'Price Standard Deviation ($/kWh)', 'RTO_ISO': 'RTO/ISO'},
        color='price',
        color_continuous_scale='Reds',
        height=600
    )
    
    fig3.update_layout(xaxis_tickangle=-45)
    fig3.write_html(os.path.join(output_dir, 'price_volatility_by_rto.html'))
    fig3.write_image(os.path.join(output_dir, 'price_volatility_by_rto.png'))
    
    # 4. Seasonal patterns by RTO/ISO
    print("Creating seasonal patterns by RTO/ISO visualization...")
    seasonal_patterns = df.groupby(['RTO_ISO', 'month'])['price'].mean().reset_index()
    
    fig4 = make_subplots(
        rows=2, 
        cols=4,
        subplot_titles=major_rtos,
        shared_yaxes=True
    )
    
    for i, rto in enumerate(major_rtos):
        row = i // 4 + 1
        col = i % 4 + 1
        
        rto_data = seasonal_patterns[seasonal_patterns['RTO_ISO'] == rto]
        
        fig4.add_trace(
            go.Scatter(
                x=rto_data['month'],
                y=rto_data['price'],
                mode='lines+markers',
                name=rto
            ),
            row=row, 
            col=col
        )
        
        # Update x-axis for each subplot
        fig4.update_xaxes(
            title_text="Month",
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            row=row,
            col=col
        )
        
        if col == 1:
            fig4.update_yaxes(title_text="Price ($/kWh)", row=row, col=col)
    
    fig4.update_layout(
        height=800,
        title_text="Seasonal Price Patterns by RTO/ISO (2020-2024)",
        showlegend=False
    )
    
    fig4.write_html(os.path.join(output_dir, 'seasonal_patterns_by_rto.html'))
    fig4.write_image(os.path.join(output_dir, 'seasonal_patterns_by_rto.png'))
    
    # 5. Box plots to show price distributions by RTO/ISO
    print("Creating price distribution by RTO/ISO visualization...")
    
    fig5 = px.box(
        df[df['RTO_ISO'].isin(major_rtos)],
        x='RTO_ISO',
        y='price',
        color='RTO_ISO',
        title='Electricity Price Distribution by RTO/ISO (2020-2024)',
        labels={'price': 'Price ($/kWh)', 'RTO_ISO': 'RTO/ISO'},
        height=600
    )
    
    fig5.update_layout(showlegend=False)
    fig5.write_html(os.path.join(output_dir, 'price_distribution_by_rto.html'))
    fig5.write_image(os.path.join(output_dir, 'price_distribution_by_rto.png'))
    
    # 6. Year-over-year price changes by RTO/ISO
    print("Creating year-over-year price changes by RTO/ISO visualization...")
    yearly_rto_prices = df.groupby(['RTO_ISO', 'year'])['price'].mean().reset_index()
    
    # Calculate year-over-year percentage changes
    yearly_changes = []
    
    for rto in major_rtos:
        rto_data = yearly_rto_prices[yearly_rto_prices['RTO_ISO'] == rto].sort_values('year')
        
        for i in range(1, len(rto_data)):
            prev_year = rto_data.iloc[i-1]['year']
            curr_year = rto_data.iloc[i]['year']
            prev_price = rto_data.iloc[i-1]['price']
            curr_price = rto_data.iloc[i]['price']
            
            pct_change = (curr_price - prev_price) / prev_price * 100
            
            yearly_changes.append({
                'RTO_ISO': rto,
                'year': curr_year,
                'from_year': prev_year,
                'to_year': curr_year,
                'pct_change': pct_change
            })
    
    yearly_changes_df = pd.DataFrame(yearly_changes)
    
    fig6 = px.bar(
        yearly_changes_df,
        x='RTO_ISO',
        y='pct_change',
        color='year',
        barmode='group',
        title='Year-over-Year Electricity Price Changes by RTO/ISO',
        labels={'pct_change': 'Percentage Change (%)', 'RTO_ISO': 'RTO/ISO', 'year': 'Year'},
        height=600
    )
    
    fig6.update_layout(xaxis_tickangle=-45)
    fig6.write_html(os.path.join(output_dir, 'yearly_changes_by_rto.html'))
    fig6.write_image(os.path.join(output_dir, 'yearly_changes_by_rto.png'))
    
    # 7. Heatmap of prices by RTO/ISO and year
    print("Creating price heatmap by RTO/ISO and year...")
    pivot_data = yearly_rto_prices[yearly_rto_prices['RTO_ISO'].isin(major_rtos)]
    pivot = pivot_data.pivot(index='RTO_ISO', columns='year', values='price')
    
    fig7 = px.imshow(
        pivot,
        labels=dict(x="Year", y="RTO/ISO", color="Price ($/kWh)"),
        x=pivot.columns,
        y=pivot.index,
        color_continuous_scale='Blues',
        aspect="auto",
        title="Electricity Price Heatmap by RTO/ISO and Year"
    )
    
    fig7.update_layout(height=600)
    
    # Add text annotations
    for y in range(len(pivot.index)):
        for x in range(len(pivot.columns)):
            value = pivot.iloc[y, x]
            fig7.add_annotation(
                x=pivot.columns[x],
                y=pivot.index[y],
                text=f"${value:.2f}",
                showarrow=False,
                font=dict(color="white" if value > 15 else "black")
            )
    
    fig7.write_html(os.path.join(output_dir, 'price_heatmap_by_rto_year.html'))
    fig7.write_image(os.path.join(output_dir, 'price_heatmap_by_rto_year.png'))
    
    print("Visualizations created and saved to the figures directory.")
    return {
        'avg_price_by_rto': fig1,
        'price_trends_by_rto': fig2,
        'price_volatility_by_rto': fig3,
        'seasonal_patterns_by_rto': fig4,
        'price_distribution_by_rto': fig5,
        'yearly_changes_by_rto': fig6,
        'price_heatmap_by_rto_year': fig7
    }

def generate_rto_analysis_report(input_file, output_file="rto_analysis_report.md"):
    """
    Generate a comprehensive analysis report comparing electricity prices across different RTOs/ISOs.
    
    Parameters:
    -----------
    input_file : str
        Path to the Excel file containing electricity data with RTO/ISO information
    output_file : str
        Path where the report will be saved (markdown format)
        
    Returns:
    --------
    list
        The report content as a list of strings
    """
    print(f"Reading data from {input_file}...")
    
    # Read the Excel file
    df = pd.read_excel(input_file)
    
    # Filter out rows not relevant to this analysis
    if 'sectorName' in df.columns:
        df = df[df['sectorName'] == 'all sectors'].copy()
    
    # Add date column
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')
    
    # Define major RTOs for focused analysis
    major_rtos = ['CAISO', 'ERCOT', 'ISO-NE', 'MISO', 'NYISO', 'PJM', 'SPP', 'WEIM']
    
    # Filter data for major RTOs only
    major_rto_data = df[df['RTO_ISO'].isin(major_rtos)].copy()
    
    # Calculate key statistics by RTO/ISO
    rto_stats = major_rto_data.groupby('RTO_ISO').agg({
        'price': ['mean', 'std', 'min', 'max'],
        'stateDescription': lambda x: ', '.join(sorted(set(x))),
        'revenue': 'sum',
        'sales': 'sum'
    }).reset_index()
    
    # Flatten multi-level columns
    rto_stats.columns = ['RTO_ISO', 'avg_price', 'price_std', 'min_price', 'max_price', 
                         'states', 'total_revenue', 'total_sales']
    
    # Calculate average price by year and RTO/ISO
    yearly_rto_prices = major_rto_data.groupby(['RTO_ISO', 'year'])['price'].mean().reset_index()
    
    # Calculate year-over-year price changes
    yearly_changes = []
    
    for rto in major_rtos:
        rto_data = yearly_rto_prices[yearly_rto_prices['RTO_ISO'] == rto].sort_values('year')
        
        for i in range(1, len(rto_data)):
            prev_year = rto_data.iloc[i-1]['year']
            curr_year = rto_data.iloc[i]['year']
            prev_price = rto_data.iloc[i-1]['price']
            curr_price = rto_data.iloc[i]['price']
            
            pct_change = (curr_price - prev_price) / prev_price * 100
            
            yearly_changes.append({
                'RTO_ISO': rto,
                'year': curr_year,
                'from_year': prev_year,
                'to_year': curr_year,
                'pct_change': pct_change
            })
    
    yearly_changes_df = pd.DataFrame(yearly_changes)
    
    # Calculate seasonal patterns
    monthly_patterns = major_rto_data.groupby(['RTO_ISO', 'month'])['price'].mean().reset_index()
    
    # Calculate average monthly fluctuation (max month avg - min month avg)
    seasonal_fluctuation = []
    for rto in major_rtos:
        rto_monthly = monthly_patterns[monthly_patterns['RTO_ISO'] == rto]
        max_month_price = rto_monthly['price'].max()
        min_month_price = rto_monthly['price'].min()
        fluctuation = max_month_price - min_month_price
        seasonal_fluctuation.append({
            'RTO_ISO': rto,
            'max_month_price': max_month_price,
            'min_month_price': min_month_price,
            'seasonal_fluctuation': fluctuation,
            'fluctuation_pct': (fluctuation / min_month_price) * 100
        })
    
    seasonal_fluctuation_df = pd.DataFrame(seasonal_fluctuation)
    
    # Generate the report
    print("Generating RTO/ISO analysis report...")
    
    # Prepare the report content
    report = []
    
    # Title
    report.append("# Electricity Price Analysis by RTO/ISO (2020-2024)\n")
    
    # Introduction
    report.append("## Introduction\n")
    report.append("This report analyzes electricity price data across different Regional Transmission Organizations (RTOs) and Independent System Operators (ISOs) in the United States from 2020 to 2024. The analysis provides insights into price variations, trends, volatility, and seasonal patterns across these different electricity markets.\n")
    
    # Overview of RTOs/ISOs
    report.append("## Overview of RTOs/ISOs\n")
    report.append("Regional Transmission Organizations (RTOs) and Independent System Operators (ISOs) are entities that coordinate, control, and monitor multi-state electric grids in the United States. They're responsible for ensuring reliability, managing wholesale electricity markets, and coordinating planning across their regions.\n")
    
    # List of analyzed RTOs/ISOs
    report.append("Major RTOs/ISOs included in this analysis:\n\n")
    
    # Prepare RTO descriptions
    rto_descriptions = {
        'CAISO': "California Independent System Operator - operates the grid for most of California",
        'ERCOT': "Electric Reliability Council of Texas - manages the grid for most of Texas (separate from other U.S. grids)",
        'ISO-NE': "ISO New England - serves six New England states: Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, and Vermont",
        'MISO': "Midcontinent Independent System Operator - covers parts of the Midwest, South, and parts of Canada",
        'NYISO': "New York Independent System Operator - serves New York State",
        'PJM': "PJM Interconnection - serves parts of 13 states in the Mid-Atlantic and Midwest regions, and the District of Columbia",
        'SPP': "Southwest Power Pool - covers parts of the central U.S. including Arkansas, Oklahoma, Kansas, and parts of surrounding states",
        'WEIM': "Western Energy Imbalance Market - a real-time market covering parts of the Western U.S., led by CAISO"
    }
    
    for rto in major_rtos:
        report.append(f"- **{rto}**: {rto_descriptions.get(rto, '')}\n")
    
    # Key Price Statistics
    report.append("\n## Key Price Statistics by RTO/ISO\n")
    report.append("The table below summarizes key electricity price statistics for each RTO/ISO during the analysis period (2020-2024):\n\n")
    
    # Prepare the statistics table
    stats_table = rto_stats.copy()
    stats_table = stats_table[['RTO_ISO', 'avg_price', 'price_std', 'min_price', 'max_price']]
    stats_table.columns = ['RTO/ISO', 'Average Price ($/kWh)', 'Price Std Dev', 'Minimum Price', 'Maximum Price']
    stats_table = stats_table.sort_values('Average Price ($/kWh)', ascending=False)
    
    # Format the values
    for col in stats_table.columns[1:]:
        stats_table[col] = stats_table[col].apply(lambda x: f"${x:.2f}")
    
    # Add the table to the report
    report.append(tabulate(stats_table, headers='keys', tablefmt='pipe', showindex=False))
    
    # Average Price Comparison
    report.append("\n\n## Average Price Comparison\n")
    report.append("There are significant variations in average electricity prices across different RTOs/ISOs:\n\n")
    
    # Get highest and lowest price RTOs
    highest_price_rto = rto_stats.loc[rto_stats['avg_price'].idxmax()]
    lowest_price_rto = rto_stats.loc[rto_stats['avg_price'].idxmin()]
    
    # Calculate the difference
    price_difference = highest_price_rto['avg_price'] - lowest_price_rto['avg_price']
    price_ratio = highest_price_rto['avg_price'] / lowest_price_rto['avg_price']
    
    report.append(f"- **Highest Average Price**: {highest_price_rto['RTO_ISO']} at ${highest_price_rto['avg_price']:.2f}/kWh\n")
    report.append(f"- **Lowest Average Price**: {lowest_price_rto['RTO_ISO']} at ${lowest_price_rto['avg_price']:.2f}/kWh\n")
    report.append(f"- **Price Difference**: ${price_difference:.2f}/kWh\n")
    report.append(f"- **Price Ratio**: Electricity in {highest_price_rto['RTO_ISO']} costs {price_ratio:.1f}x more than in {lowest_price_rto['RTO_ISO']}\n")
    
    # Price Trends
    report.append("\n## Price Trends (2020-2024)\n")
    report.append("Analysis of year-over-year price changes for each RTO/ISO reveals interesting patterns:\n\n")
    
    # Calculate average annual price change by RTO
    avg_annual_change = yearly_changes_df.groupby('RTO_ISO')['pct_change'].mean().reset_index()
    avg_annual_change = avg_annual_change.sort_values('pct_change', ascending=False)
    
    report.append("### Average Annual Price Change by RTO/ISO\n\n")
    
    for _, row in avg_annual_change.iterrows():
        direction = "increase" if row['pct_change'] > 0 else "decrease"
        report.append(f"- **{row['RTO_ISO']}**: {abs(row['pct_change']):.1f}% average annual {direction}\n")
    
    report.append("\n### Yearly Price Changes by RTO/ISO\n\n")
    
    # Create a summary of year-by-year changes
    for year in sorted(yearly_changes_df['year'].unique()):
        report.append(f"#### {year} Price Changes\n")
        year_data = yearly_changes_df[yearly_changes_df['year'] == year].sort_values('pct_change', ascending=False)
        
        for _, row in year_data.iterrows():
            direction = "increase" if row['pct_change'] > 0 else "decrease"
            report.append(f"- **{row['RTO_ISO']}**: {abs(row['pct_change']):.1f}% {direction}\n")
        
        report.append("\n")
    
    # Price Volatility
    report.append("## Price Volatility\n")
    report.append("Price volatility is an important factor for consumers and market participants. The standard deviation of prices can be used as a measure of volatility.\n\n")
    
    # Prepare volatility data
    volatility_data = rto_stats[['RTO_ISO', 'avg_price', 'price_std']].copy()
    volatility_data['coefficient_of_variation'] = volatility_data['price_std'] / volatility_data['avg_price'] * 100
    volatility_data = volatility_data.sort_values('coefficient_of_variation', ascending=False)
    
    report.append("### Volatility Ranking (Coefficient of Variation)\n\n")
    
    for _, row in volatility_data.iterrows():
        report.append(f"- **{row['RTO_ISO']}**: {row['coefficient_of_variation']:.1f}% (Std Dev: ${row['price_std']:.2f})\n")
    
    # Seasonal Patterns
    report.append("\n## Seasonal Price Patterns\n")
    report.append("Electricity prices often follow seasonal patterns due to changes in demand and supply conditions throughout the year.\n\n")
    
    # Add seasonal fluctuation data
    seasonal_fluctuation_df = seasonal_fluctuation_df.sort_values('fluctuation_pct', ascending=False)
    
    report.append("### Seasonal Price Fluctuation\n\n")
    
    for _, row in seasonal_fluctuation_df.iterrows():
        report.append(f"- **{row['RTO_ISO']}**: {row['fluctuation_pct']:.1f}% fluctuation (${row['min_month_price']:.2f} to ${row['max_month_price']:.2f})\n")
    
    # Calculate seasonal patterns - when prices peak for each RTO
    peak_months = {}
    for rto in major_rtos:
        rto_monthly = monthly_patterns[monthly_patterns['RTO_ISO'] == rto]
        peak_month = rto_monthly.loc[rto_monthly['price'].idxmax()]['month']
        peak_months[rto] = peak_month
    
    # Convert month numbers to names
    month_names = {
        1: 'January', 2: 'February', 3: 'March', 4: 'April', 
        5: 'May', 6: 'June', 7: 'July', 8: 'August',
        9: 'September', 10: 'October', 11: 'November', 12: 'December'
    }
    
    report.append("\n### Peak Price Months by RTO/ISO\n\n")
    
    for rto, month in peak_months.items():
        report.append(f"- **{rto}**: {month_names[month]}\n")
    
    # Market Comparisons
    report.append("\n## Market Comparisons\n")
    
    # Calculate the correlation matrix between RTO prices
    monthly_pivoted = major_rto_data.pivot_table(
        index='date',
        columns='RTO_ISO',
        values='price',
        aggfunc='mean'
    )
    
    correlation_matrix = monthly_pivoted.corr()
    
    # Find the most and least correlated markets
    most_correlated = None
    least_correlated = None
    max_corr = -1
    min_corr = 2
    
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            rto1 = correlation_matrix.columns[i]
            rto2 = correlation_matrix.columns[j]
            corr = correlation_matrix.iloc[i, j]
            
            if corr > max_corr:
                max_corr = corr
                most_correlated = (rto1, rto2)
            
            if corr < min_corr:
                min_corr = corr
                least_correlated = (rto1, rto2)
    
    report.append("### Price Correlation Between Markets\n\n")
    
    report.append(f"- **Most correlated markets**: {most_correlated[0]} and {most_correlated[1]} (correlation: {max_corr:.2f})\n")
    report.append(f"- **Least correlated markets**: {least_correlated[0]} and {least_correlated[1]} (correlation: {min_corr:.2f})\n")
    
    # Final Insights
    report.append("\n## Key Insights\n")
    report.append("Based on the analysis of electricity prices across different RTOs/ISOs from 2020 to 2024, several key insights emerge:\n\n")
    
    # Add insights based on the analysis
    insights = [
        f"**Price Disparities**: There are significant price disparities across regions, with {highest_price_rto['RTO_ISO']} having {price_ratio:.1f}x higher average prices than {lowest_price_rto['RTO_ISO']}.",
        f"**Price Trends**: {avg_annual_change.iloc[0]['RTO_ISO']} has experienced the largest average annual price increases at {avg_annual_change.iloc[0]['pct_change']:.1f}%.",
        f"**Volatility**: {volatility_data.iloc[0]['RTO_ISO']} shows the highest price volatility, while {volatility_data.iloc[-1]['RTO_ISO']} demonstrates the most stable prices.",
        f"**Seasonal Patterns**: {seasonal_fluctuation_df.iloc[0]['RTO_ISO']} exhibits the most pronounced seasonal price fluctuations, with prices varying by {seasonal_fluctuation_df.iloc[0]['fluctuation_pct']:.1f}% throughout the year.",
        f"**Market Interconnection**: {most_correlated[0]} and {most_correlated[1]} show highly correlated price movements, suggesting similar market drivers or interconnected grid operations."
    ]
    
    for insight in insights:
        report.append(f"- {insight}\n")
    
    # Conclusion
    report.append("\n## Conclusion\n")
    report.append("This analysis highlights the complex nature of electricity markets across different regions of the United States. Factors such as regional fuel mix, transmission constraints, market design, regulatory frameworks, and weather patterns all contribute to the observed differences in electricity prices across RTOs/ISOs.\n\n")
    report.append("Understanding these differences and patterns is crucial for market participants, policymakers, regulators, and consumers to make informed decisions about energy consumption, investment, and policy development.\n")
    
    # Write the report to a file
    with open(output_file, 'w') as f:
        f.write('\n'.join(report))
    
    print(f"Report generated and saved to {output_file}")
    return report


def prepare_dashboard_data(df, output_dir):
    """
    Prepare data files for a dashboard application by extracting and
    transforming relevant data from the electricity dataset.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing electricity data with RTO/ISO information
    output_dir : str
        Directory where dashboard data files will be saved
    """
    print("Preparing data for dashboard...")
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Filter for all sectors data
    if 'sectorName' in df.columns:
        all_sectors = df[df['sectorName'] == 'all sectors'].copy()
    else:
        all_sectors = df.copy()
    
    # Add date column
    all_sectors['date'] = pd.to_datetime(all_sectors['year'].astype(str) + '-' + 
                                         all_sectors['month'].astype(str) + '-01')
    
    # 1. RTO/ISO average prices
    rto_avg_prices = all_sectors.groupby('RTO_ISO')['price'].agg(
        ['mean', 'std', 'min', 'max', 'count']
    ).reset_index()
    rto_avg_prices.columns = ['rto_iso', 'avg_price', 'std_price', 'min_price', 'max_price', 'count']
    rto_avg_prices.to_csv(os.path.join(output_dir, 'rto_avg_prices.csv'), index=False)
    
    # 2. Monthly prices by RTO/ISO
    monthly_rto_prices = all_sectors.groupby(['RTO_ISO', 'year', 'month']).agg({
        'price': 'mean',
        'revenue': 'sum',
        'sales': 'sum'
    }).reset_index()
    monthly_rto_prices['date'] = pd.to_datetime(
        monthly_rto_prices['year'].astype(str) + '-' + 
        monthly_rto_prices['month'].astype(str) + '-01'
    )
    monthly_rto_prices.to_csv(os.path.join(output_dir, 'monthly_rto_prices.csv'), index=False)
    
    # 3. State to RTO/ISO mapping
    state_rto_mapping = all_sectors[['stateDescription', 'RTO_ISO']].drop_duplicates()
    state_rto_mapping.columns = ['state', 'rto_iso']
    state_rto_mapping.to_csv(os.path.join(output_dir, 'state_rto_mapping.csv'), index=False)
    
    # 4. Yearly summary by RTO/ISO
    yearly_rto_summary = all_sectors.groupby(['RTO_ISO', 'year']).agg({
        'price': 'mean',
        'revenue': 'sum',
        'sales': 'sum',
        'customers': 'mean'
    }).reset_index()
    yearly_rto_summary.to_csv(os.path.join(output_dir, 'yearly_rto_summary.csv'), index=False)
    
    # 5. Price comparison data (latest available month)
    latest_date = all_sectors['date'].max()
    latest_prices = all_sectors[all_sectors['date'] == latest_date].copy()
    latest_prices = latest_prices[['stateDescription', 'RTO_ISO', 'price']]
    latest_prices.columns = ['state', 'rto_iso', 'price']
    latest_prices.to_csv(os.path.join(output_dir, 'latest_prices.csv'), index=False)
    
    # 6. Dashboard metadata
    major_rtos = ['CAISO', 'ERCOT', 'ISO-NE', 'MISO', 'NYISO', 'PJM', 'SPP', 'WEIM']
    dashboard_meta = {
        'data_updated': datetime.now().strftime('%Y-%m-%d'),
        'date_range': {
            'start': all_sectors['date'].min().strftime('%Y-%m-%d'),
            'end': all_sectors['date'].max().strftime('%Y-%m-%d')
        },
        'major_rtos': major_rtos,
        'available_metrics': ['price', 'revenue', 'sales', 'customers'],
        'total_records': len(all_sectors)
    }
    
    # Save metadata as JSON
    import json
    with open(os.path.join(output_dir, 'dashboard_meta.json'), 'w') as f:
        json.dump(dashboard_meta, f, indent=2)
    
    print(f"Dashboard data prepared and saved to {output_dir}")

if __name__ == "__main__":
    # Run the entire analysis pipeline
    run_rto_analysis_pipeline(input_file='monthly_all_states_2020_2024.xlsx', output_dir='rto_analysis' )

Step 1: Adding RTO/ISO mapping to the data...
Reading electricity data from monthly_all_states_2020_2024.xlsx...
Loaded 3038 rows of data
Adding RTO/ISO information...

Number of entries by RTO/ISO:
RTO_ISO
PJM         588
MISO        490
WEIM        441
Multiple    441
Other       343
ISO-NE      343
SPP         245
CAISO        49
NYISO        49
ERCOT        49
Name: count, dtype: int64

Saving modified data to rto_analysis/data/monthly_all_states_with_rto.xlsx...

Step 2: Generating visualizations...
Reading data from rto_analysis/data/monthly_all_states_with_rto.xlsx...
Creating average price by RTO/ISO visualization...


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [3]:

import gurobipy as gp
from gurobipy import GRB
import numpy as np

def green_llm_optimization(params):
    """
    Implement the Green-LLM optimization model using Gurobi
    
    Args:
        params: Dictionary containing all necessary parameters
        
    Returns:
        model: Solved Gurobi model
        solution: Dictionary containing optimal decision variables
    """
    # Extract parameters
    I = params['I']  # Set of user areas
    J = params['J']  # Set of data centers
    K = params['K']  # Set of query types
    T = params['T']  # Set of time slots
    R = params['R']  # Set of resource types
    DeltaT = params['DeltaT']  # Set of time intervals

    
    # Query parameters
    lmbda = params['lambda']      # Number of type k queries from area i at time t
    h = params['h']               # Average input token count for type k queries
    f = params['f']               # Average output token count for type k queries
    
    # Energy parameters
    tau_in = params['tau_in']     # Energy consumption coefficient for input tokens
    tau_out = params['tau_out']   # Energy consumption coefficient for output tokens
    c = params['c']               # Local marginal price at DC j at time t
    theta = params['theta']       # Carbon intensity at DC j
    delta = params['delta']       # Carbon tax for DC j at time t
    
    # Data center parameters
    P_w = params['P_w']          # Renewable energy generated at DC j at time t
    P_max = params['P_max']      # Maximum power from grid for DC j at time t
    PUE = params['PUE']          # Power Usage Effectiveness of DC j

    # Resource parameters
    C = params['C']              # Capacity of type r resource at DC j
    alpha = params['alpha']      # Type r resource required for processing type k tokens
    
    # Model placement parameters
    z_prev = params['z_prev']    # Known placement status from previous time slot
    f_download = params['f_download']  # Submodel download cost at DC j
    
    # QoS parameters
    s = params['s']              # Unmet penalty for type k query in area i at time t
    Gamma = params['Gamma']      # Maximum allowed unmet demand ratio for area i
    e = params['e']              # Error rate for processing type k queries
    E = params['E']              # Minimum required accuracy for type k queries
    
    # Delay parameters
    beta = params['beta']        # Average token size at time t
    B = params['B']              # Available bandwidth between i and j
    d = params['d']              # Network delay between area i and DC j
    v = params['v']              # Processing delay at DC j at time t
    Delta = params['Delta']      # Threshold on average round delay
    
    # Water parameters
    WUE = params['WUE']          # Water Usage Effectiveness of DC j at time t
    EWIF = params['EWIF']        # Energy-Water Intensity Factor of DC j at time t
    Z = params['Z']              # Maximum allowed water consumption
    
    # Create a new model
    model = gp.Model("Green-LLM")
    
    # Create decision variables using addVars for cleaner implementation
    x = model.addVars([(i, j, k, t) for i in I for j in J for k in K for t in T], vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x")
    P_g = model.addVars([(j, t) for j in J for t in T], vtype=GRB.CONTINUOUS, lb=0, name="P_g")
    z = model.addVars([(j, k, t) for j in J for k in K for t in T], vtype=GRB.BINARY, name="z")
    q = model.addVars([(i, k, t) for i in I for k in K for t in T], vtype=GRB.CONTINUOUS, lb=0, ub=1, name="q")
    
    # Create variables for each type of delay
    D_tran = model.addVars([(i, k, t) for i in I for k in K for t in T], 
                          vtype=GRB.CONTINUOUS, lb=0, name="D_tran")
    D_prop = model.addVars([(i, k, t) for i in I for k in K for t in T], 
                          vtype=GRB.CONTINUOUS, lb=0, name="D_prop")
    D_proc = model.addVars([(i, k, t) for i in I for k in K for t in T], 
                          vtype=GRB.CONTINUOUS, lb=0, name="D_proc")
    
    # Water consumption variable
    woc = model.addVars([(j, t) for j in J for t in T], vtype=GRB.CONTINUOUS, lb=0, name="woc")
    
    # Helper variables
    eta = model.addVars([(j, t) for j in J for t in T], vtype=GRB.CONTINUOUS, lb=0, name="eta")
    P_d = model.addVars([(j, t) for j in J for t in T], vtype=GRB.CONTINUOUS, lb=0, name="P_d")
    P_c = model.addVars([(j,t) for j in J for t in T], vtype=GRB.CONTINUOUS, lb=0, name="P_c")
    # Update model to integrate new variables
    model.update()
    
    # Objective function components
    
    # C1: Energy consumption cost
    # C1 = gp.quicksum((tau_in[k] * h[k] * lmbda[i, k, t] * x[i, j, k, t] + tau_out[k] * f[k] * lmbda[i, k, t] * x[i, j, k, t]) * gamma[j, t] for i in I for j in J for k in K for t in T)
    
    # C1: Power procurement cost
    C1 = gp.quicksum(c[j, t] * P_g[j, t] for j in J for t in T)
    
    # C3: Carbon tax
    C2 = gp.quicksum(delta[j, t] * theta[j] * P_g[j, t] for j in J for t in T)
    
    # C4: Model placement and storage cost (avoiding redundant downloads)
    C3_terms = []
    for j in J:
        for k in K:
            for t in T:
                if t == 0:
                    # For the first time slot, use the provided z_prev
                    C3_terms.append(f_download[j] * (1 - z_prev[j, k, t]) * z[j, k, t])
                else:
                    # For subsequent time slots, use the decision from previous time
                    C3_terms.append(f_download[j] * (1 - z[j, k, t-1]) * z[j, k, t])
    
    C3 = gp.quicksum(C3_terms)
    
    # C5: Unmet demand penalty
    C4 = gp.quicksum(s[i, k, t] * lmbda[i, k, t] * q[i, k, t] for i in I for k in K for t in T)
    
    # Set the objective (minimize total cost)
    model.setObjective(C1 + C2 + C3 + C4, GRB.MINIMIZE)
    
    # Computational energy consumption
    model.addConstrs((P_c[j,t] == gp.quicksum((tau_in[k] * h[k] + tau_out[k] * f[k]) * lmbda[i, k, t] * x[i, j, k, t] for i in I for k in K) for j in J for t in T), name="computation_consumption")
    
    # Total Energy consumption (including cooling and other overheads)
    model.addConstrs((P_d[j, t] == PUE[j] * P_c[j, t] for j in J for t in T), name="tot_power_consumption")

    # Power balance constraints (Equation 7)
    model.addConstrs((P_d[j, t] <= P_g[j, t] + P_w[j, t] for j in J for t in T), name="power_balance")
    
    # Grid capacity condition (Equation 8)
    model.addConstrs((P_g[j, t] <= P_max[j, t] for j in J for t in T), name="grid_capacity")
    
    # Long-term water footprint constraints (Equation 10)
    model.addConstrs((woc[j, t] == (WUE[j, t] / PUE[j] + EWIF[j, t]) * P_d[j, t] for j in J for t in T),name="water_consumption")
    model.addConstr(gp.quicksum(woc[j, t] for j in J for t in DeltaT) <= Z, name="long_term_water_footprint")
    
    
    # Ensure workload allocation only if model is placed (Equation 11a)
    # since z[j,k,t]=1 means the model IS placed at DC j
    model.addConstrs((x[i, j, k, t] <= z[j, k, t] for i in I for j in J for k in K for t in T),name="model_placement")
    
    # Ensure resource capacity is not exceeded (Equation 11b)
    model.addConstrs((gp.quicksum(alpha[k, r] * f[k] * lmbda[i, k, t] * x[i, j, k, t] for i in I for k in K) <=  C[r, j] for j in J for r in R for t in T),name="resource_capacity")
    
    # Workload allocation constraints (Equation 12)
    model.addConstrs((q[i, k, t] + gp.quicksum(x[i, j, k, t] for j in J) == 1 for i in I for k in K for t in T), name="workload_allocation")
    
    # Define transmission delay (Equation 13)
    for i in I:
        for k in K:
            for t in T:
                # Add a small epsilon to avoid division by zero in bandwidth
                model.addConstr(
                    D_tran[i, k, t] == gp.quicksum(
                        beta[i, k, t] * lmbda[i, k, t] * x[i, j, k, t] * (h[k] + f[k]) / max(0.001, B[i, j])
                        for j in J
                    ),
                    f"trans_delay_def_{i}_{k}_{t}"
                )

    # Define propagation delay (Equation 14)
    model.addConstrs((D_prop[i, k, t] == gp.quicksum(d[i, j] * lmbda[i, k, t] * x[i, j, k, t] * (h[k] + f[k]) for j in J) for i in I for k in K for t in T), name="prop_delay_def")

    # Define processing delay (Equation 15)
    model.addConstrs((D_proc[i, k, t] == gp.quicksum(v[j, t] * f[k] * lmbda[i, k, t] * x[i, j, k, t] for j in J) for i in I for k in K for t in T), name="proc_delay_def")

    # Total delay constraint (Equation 16)
    # model.addConstrs((D_tran[i, k, t] + D_prop[i, k, t] + D_proc[i, k, t] <=  Delta[i, k, t] for i in I for k in K for t in T), name="max_delay")
    
   #  QoS constraints (Equation 17)
    # model.addConstrs(
    #     (q[i, k, t] <= Gamma[i] for i in I for k in K for t in T),
    #     name="qos"
    # )
    
    # Accuracy constraints (Equation 18)
    # Modified to handle potential infeasibility - make sure accuracy requirement is proportional to satisfied demand
    for k in K:
        for t in T:
            total_demand = gp.quicksum(lmbda[i, k, t] for i in I)
            satisfied_demand = gp.quicksum((1 - q[i, k, t]) * lmbda[i, k, t] for i in I)
            model.addConstr(gp.quicksum((1 - e[k]) * lmbda[i, k, t] * x[i, j, k, t] for i in I for j in J) >= E[k] * satisfied_demand,
                f"accuracy_{k}_{t}"
            )
    
    # Set optimization parameters
    model.setParam('OutputFlag', 0)        # Display optimization details
    model.setParam('TimeLimit', 600)       # 10-minute time limit
    model.setParam('MIPGap', 0.05)         # Accept solutions within 5% of optimal
    model.setParam('FeasibilityTol', 1e-5) # Slightly relaxed feasibility tolerance
    model.setParam('NumericFocus', 3)      # Maximum numeric stability
    
    # Optional: Add warm start if available from previous solution
    if 'previous_solution' in params:
        prev_sol = params['previous_solution']
        for i in I:
            for j in J:
                for k in K:
                    for t in T:
                        if (i, j, k, t) in prev_sol['x']:
                            x[i, j, k, t].start = prev_sol['x'][i, j, k, t]
    
    # Optimize the model
    try:
        model.optimize()
    except gp.GurobiError as e:
        print(f"Optimization error: {e}")
        return model, None
    
    # Check if optimal solution found
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT or model.status == GRB.SUBOPTIMAL:
        # Extract solution
        solution = {
            'x': {(i, j, k, t): x[i, j, k, t].X for i in I for j in J for k in K for t in T},
            'P_g': {(j, t): P_g[j, t].X for j in J for t in T},
            'P_c': {(j, t): P_c[j, t].X for j in J for t in T},
            'P_d': {(j, t): P_d[j, t].X for j in J for t in T},
            'z': {(j, k, t): z[j, k, t].X for j in J for k in K for t in T},
            'q': {(i, k, t): q[i, k, t].X for i in I for k in K for t in T},
            'objective_value': model.objVal,
            'objective_components': {
                'C1': C1.getValue(),
                'C2': C2.getValue(),
                'C3': C3.getValue(),
                'C4': C4.getValue(),
            },
            'status': model.status
        }
        
        # Add solution quality metrics
        if model.status == GRB.OPTIMAL:
            solution['quality'] = "Optimal"
            print(f"Optimization status: {solution['quality']}")
            print(f"Objective value: {solution['objective_value']:.2f}")
            print(f"Energy cost: {solution['objective_components']['C1']:.2f}")
        elif model.status == GRB.TIME_LIMIT:
            solution['quality'] = f"Time limit reached. Gap: {model.MIPGap*100:.2f}%"
        elif model.status == GRB.SUBOPTIMAL:
            solution['quality'] = "Suboptimal solution found"
            
        return model, solution
    else:
        status_codes = {
            GRB.LOADED: "Model is loaded, but not solved",
            GRB.INFEASIBLE: "Model is infeasible",
            GRB.INF_OR_UNBD: "Model is infeasible or unbounded",
            GRB.UNBOUNDED: "Model is unbounded",
            GRB.ITERATION_LIMIT: "Iteration limit reached",
            GRB.NODE_LIMIT: "Node limit reached",
            GRB.SOLUTION_LIMIT: "Solution limit reached",
            GRB.INTERRUPTED: "Optimization was interrupted",
            GRB.INPROGRESS: "Optimization in progress",
            GRB.USER_OBJ_LIMIT: "User objective limit reached"
        }
        
        status_msg = status_codes.get(model.status, f"Unknown status: {model.status}")
        print(f"Optimization failed: {status_msg}")
        
        # If infeasible, try to find the source of infeasibility
        if model.status == GRB.INFEASIBLE:
            print("Computing IIS (Irreducible Inconsistent Subsystem)...")
            model.computeIIS()
            model.write("model_iis.ilp")
            print("IIS written to model_iis.ilp")
            
        return model, None
    
# Run the optimization
model, solution = green_llm_optimization(params)



NameError: name 'params' is not defined

In [42]:
# Generate temporal-spatial demand for queries
I = list(range(10))
J = list(range(10))   
K = list(range(5))
T = list(range(24))
lmbda = {}
for i in I:
    for k in K:
        for t in T:
            # Generate demand between 30-50 queries per time slot
            lmbda[(i, k, t)] = np.random.randint(30, 51)
    
params['lambda'] = lmbda

# Print some sample demand values to verify
print("\nSample demand (lambda) values:")
for i in I:  
    for k in K:  
        for t in T:  # Every 8 hours
            print(f"Region {i}, Query type {k}, Hour {t}: {lmbda[(i, k, t)]} queries")

# Calculate total demand metrics
total_demand = sum(lmbda.values())
avg_demand_per_hour = total_demand / (len(I) * len(K) * len(T))



Sample demand (lambda) values:
Region 0, Query type 0, Hour 0: 44 queries
Region 0, Query type 0, Hour 1: 45 queries
Region 0, Query type 0, Hour 2: 40 queries
Region 0, Query type 0, Hour 3: 35 queries
Region 0, Query type 0, Hour 4: 40 queries
Region 0, Query type 0, Hour 5: 32 queries
Region 0, Query type 0, Hour 6: 44 queries
Region 0, Query type 0, Hour 7: 30 queries
Region 0, Query type 0, Hour 8: 36 queries
Region 0, Query type 0, Hour 9: 30 queries
Region 0, Query type 0, Hour 10: 46 queries
Region 0, Query type 0, Hour 11: 38 queries
Region 0, Query type 0, Hour 12: 42 queries
Region 0, Query type 0, Hour 13: 40 queries
Region 0, Query type 0, Hour 14: 49 queries
Region 0, Query type 0, Hour 15: 35 queries
Region 0, Query type 0, Hour 16: 30 queries
Region 0, Query type 0, Hour 17: 36 queries
Region 0, Query type 0, Hour 18: 41 queries
Region 0, Query type 0, Hour 19: 49 queries
Region 0, Query type 0, Hour 20: 48 queries
Region 0, Query type 0, Hour 21: 46 queries
Region 0, 